In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3-8b-medical-assistant-v3/transformers/default/1/llama-3-8b-medical-assistant-v3/adapter_model.safetensors
/kaggle/input/llama-3-8b-medical-assistant-v3/transformers/default/1/llama-3-8b-medical-assistant-v3/adapter_config.json
/kaggle/input/llama-3-8b-medical-assistant-v3/transformers/default/1/llama-3-8b-medical-assistant-v3/README.md
/kaggle/input/llama-3-8b-medical-assistant-v3/transformers/default/1/llama-3-8b-medical-assistant-v3/checkpoint-630/adapter_model.safetensors
/kaggle/input/llama-3-8b-medical-assistant-v3/transformers/default/1/llama-3-8b-medical-assistant-v3/checkpoint-630/trainer_state.json
/kaggle/input/llama-3-8b-medical-assistant-v3/transformers/default/1/llama-3-8b-medical-assistant-v3/checkpoint-630/training_args.bin
/kaggle/input/llama-3-8b-medical-assistant-v3/transformers/default/1/llama-3-8b-medical-assistant-v3/checkpoint-630/adapter_config.json
/kaggle/input/llama-3-8b-medical-assistant-v3/transformers/default/1/llama-3-8b-medical-assist

#### Tuned model merging.

We have already trained the version 2 of the model, and now it should be merged with the version 1 model to be saved and used in the project.

In [2]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

Logging in to Hugging Face.

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("huggingface_token")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Creating variables for base and new models.

In [4]:
base_model = "/kaggle/input/llama-3-8b-medical-assistant-v2.5/transformers/default/1/llama-3-8b-medical-assistant-v2"
new_model = "/kaggle/input/llama-3-8b-medical-assistant-v3/transformers/default/1/llama-3-8b-medical-assistant-v3"

To merge the base model with the adapter we should load the tokenizer and base model using the transformers library. Then, we set up the chat format using the trl library. Finally, we load and merge the adapter to the base model using the PEFT library.

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format

# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

2024-07-30 20:11:35.075544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 20:11:35.075684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 20:11:35.222225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
torch.cuda.empty_cache()

In [7]:
# Merge adapter with base model
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
model = PeftModel.from_pretrained(base_model_reload, new_model)

In [8]:
model = model.merge_and_unload()

#### Model Inference.

To verify if our model has been merged correctly, we perform a simple inference using pipeline from the transformers library. We convert the message using the chat template and then provide a prompt to the pipeline. The pipeline was initialized using the model, tokenizer, and task type.

In [9]:
messages = [{
    "role": "user",
    "content": """
        Hello Doctor, please provide me with a brief summary of this medicine leaflet, 
        from 3 to 6 most important side effects, important information, and recommendations in 250 words maximum:
        What you should know about Distalgesic tablets
        Distalgesic    Please read this leaflet carefully before you start to take your medicine.  
        It does not contain all the information about your medicine that you may need to know so if you 
        have any questions, ask your doctor or pharmacist. This leaflet only applies to Distalgesic tablets.    
        What is in your medicine?   Your medicine is called Distalgesic, which is a brand of co-proxamol.  
        Its active ingredients are dextropropoxyphene hydrochloride and paracetamol.  
        Each white, pillow-shaped tablet contains 32.5mg dextropropoxyphene hydrochloride 
        and 325mg paracetamol.  They are labelled DG.  The tablets also contain the inactive 
        ingredients maize starch and magnesium stearate, and are coated with hydroxypropyl 
        methyl cellulose 15, glycerine and titanium dioxide.  They come in blister packs of 100.   
        Distalgesic is one of a group of medicines called analgesics that help to ease pain.   
        Distalgesic is made by Eli Lilly and Company Limited, Kingsclere Road, Basingstoke, 
        Hampshire, RG21 6XA.  The product licence is held by Dista Products Limited, 
        Dextra Court, Chapel Hill, Basingstoke, Hampshire RG21 5SY     
        Why Distalgesic?   Distalgesic is given to help reduce mild to moderate pain.     
        Before taking your medicine    Make sure it is sate for you to take Distalgesic. 
        It you answer YES to any of the following questions or you are not sure about 
        the answer, tell your doctor or pharmacist.      
        Have you ever been allergic to this or any other pain-killing medicine?       
        Do you suffer from depression or any other psychiatric illness?     
        Have you ever been an alcoholic or drug addict?     
        Are you taking any other medicine, especially 'blood thinner' 
        medicine (such as warfarin), antidepressants or drugs for epilepsy?     
        Are you pregnant or could you be?     Are you breast-feeding?       
        Have you ever had problems with your liver or kidneys?      
        Do not take more Distalgesic than your doctor tells you to  .    
        Do not drink alcohol while taking Distalgesic.    
        It can be very dangerous.  Discuss this with your doctor if you have any questions.   
        If you take drugs such as antihistamines, tranquillisers or sleeping tablets 
        when you are taking Distalgesic, you may become extremely sleepy.   
        Distalgesic on its own makes some people very sleepy, especially when 
        they first start taking it.   Do not drive or use dangerous machinery 
        if you feel sleepy.   Do not give Distalgesic to children.      
        Please read the back of this leaflet          
        What you should know about Distalgesic tablets - continued         
        Taking your medicine     Follow your doctor’s instructions.  
        Check the label to see how many tablets to take and how often to take them.      
        Usual dose: take 2 tablets three or four times a day.       
        Do not take more than 8 tablets in 24 hours.    
        If you are elderly, you may need to take fewer.  
        If you are not sure how many tablets to take, ask your doctor or pharmacist     
        Swallow the tablets with a drink of water.     
        Do not take more tablets than your doctor tells you to.  
        You can become addicted to these tablets if you take too many for a long time.     
        If you miss a dose, take another dose as soon as you can.  Then go on as before.  
        Never double up on the next dose to make up for the one missed.      
        OVERDOSE: If you, or anyone else, ever take too many Distalgesic tablets, 
        dial 999 for an ambulance straight away.  Then phone your doctor.    
        An overdose can damage your liver or cause coma, convulsions or death.       
        While taking your medicine     Distalgesic tablets may cause side-effects. 
        These include feeling dizzy or tired, feeling sick and vomiting. 
        They will often go away if you lie down.  Tell your doctor if they are troublesome. 
        Very occasionally, Distalgesic may give you a rash, headaches or stomach ache. 
        It might also make you constipated or moody, or give you problems with your sight, 
        hallucinations or jaundice.  If you have these or any other ill-effects that are 
        troublesome, tell your doctor.       How to store your medicine     
        Do not use Distalgesic after the 'Use Before' date.   
        Keep your tablets at room temperature, in a dry, safe place where children 
        cannot see or reach them.  Your tablets could harm them.   
        When your treatment is over, we suggest that you take any tablets you have 
        left back to the pharmacist who will dispose of them safely.     
        REMEMBER: This medicine is for you.  Only a doctor can prescribe it.  
        Never give it to others.  It may harm them, even if their symptoms are 
        the same as yours.   'Distalgesic' is a registered trademark of Dista
        Products Limited.   Date of Leaflet Preparation: November 1995 © Dista 
        Products Limited, 1995.
        """
}]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=2.1, top_k=50, top_p=0.95)
print(outputs)

[{'generated_text': "<|im_start|>user\n\n        Hello Doctor, please provide me with a brief summary of this medicine leaflet, \n        from 3 to 6 most important side effects, important information, and recommendations in 250 words maximum:\n        What you should know about Distalgesic tablets\n        Distalgesic    Please read this leaflet carefully before you start to take your medicine.  \n        It does not contain all the information about your medicine that you may need to know so if you \n        have any questions, ask your doctor or pharmacist. This leaflet only applies to Distalgesic tablets.    \n        What is in your medicine?   Your medicine is called Distalgesic, which is a brand of co-proxamol.  \n        Its active ingredients are dextropropoxyphene hydrochloride and paracetamol.  \n        Each white, pillow-shaped tablet contains 32.5mg dextropropoxyphene hydrochloride \n        and 325mg paracetamol.  They are labelled DG.  The tablets also contain the inact

We can see that the fine-tuned model is working as expected after being merged.

#### Saving and pushing the merged model.

In [10]:
model.save_pretrained("llama-3-8b-medical-assistant-v3")
tokenizer.save_pretrained("llama-3-8b-medical-assistant-v3")

('llama-3-8b-medical-assistant-v3/tokenizer_config.json',
 'llama-3-8b-medical-assistant-v3/special_tokens_map.json',
 'llama-3-8b-medical-assistant-v3/tokenizer.json')

In [11]:
model.push_to_hub("llama-3-8b-medical-assistant-v3", use_temp_dir=False)
tokenizer.push_to_hub("llama-3-8b-medical-assistant-v3", use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/McSimoff/llama-3-8b-medical-assistant-v3/commit/cc2c5babff52b060136c9ae136baf7a01106d117', commit_message='Upload tokenizer', commit_description='', oid='cc2c5babff52b060136c9ae136baf7a01106d117', pr_url=None, pr_revision=None, pr_num=None)